<a href="https://colab.research.google.com/github/supremmm/FreeCodeCamp_machine_learning_certification/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Import data
#!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# statistics and visualisation
dataset.describe()
dataset.hist(bins=30, figsize=(12,8))
plt.tight_layout()
plt.show()

In [ ]:
# desciption of the dataset
dataset.info()



*   sex, smoker, and region are categorical
*   we will convert them to numbers



# data processing

In [ ]:
cat = ["sex", "smoker", "region"]
for i in cat:
  print("\n", dataset[i].value_counts())

In [ ]:
df = dataset.copy()
df.head()

In [ ]:
# look on correlations
from pandas.plotting import scatter_matrix

attributes = ["age", "bmi", "children", "expenses"]
scatter_matrix(df[attributes], figsize=(12,8))
plt.show()

In [ ]:
#let us focus on bmi/expenses
df.plot(kind="scatter", x="bmi", y="expenses", alpha=0.1)

In [ ]:
smooker = dataset[dataset["smoker"] == "yes"]
non_smoker = dataset[dataset["smoker"] == "no"]

#male = dataset[(dataset["sex"] == "male") & (dataset["smoker"] == "yes")]
#female = dataset[dataset["sex"] == "female"]

smooker.plot(kind="scatter", x="bmi", y="expenses", alpha=0.1, title="smoker")
non_smoker.plot(kind="scatter", x="bmi", y="expenses", alpha=0.1, title= "non smoker")

male_smoker = smooker[smooker["sex"] == "male"]
female_smoker = smooker[smooker["sex"] == "female"]

male_smoker.plot(kind="scatter", x="bmi", y="expenses", alpha=0.1, title="male smoker")
female_smoker.plot(kind="scatter", x="bmi", y="expenses", alpha=0.1, title="female smoker")

In [ ]:
# convert the categorial columns into binary indicator columns
df = pd.get_dummies(df, columns=['sex', 'smoker', 'region'], drop_first=True, dtype=float)
df.head()



In [ ]:

df['bmi_smoker'] = df['bmi'] * df['smoker_yes']
df['age_children'] = df['age'] * df['children']


In [ ]:
# High-BMI smokers incur disproportionate expenses
from pandas.plotting import scatter_matrix

list=["bmi_smoker", "expenses"]
scatter_matrix(df[list], figsize=(12,8))
plt.show()

In [ ]:
# use 80% of the data as the train_dataset and 20% as the test_dataset
train_dataset = df.sample(frac=0.8, random_state= 0)
test_dataset = df.drop(train_dataset.index)


In [ ]:

# pop off the "espenses" column from these dataset to create new datasets called train_labels and tetst_labels
# use the label when training the model
train_labels = train_dataset.pop("expenses")
test_labels = test_dataset.pop("expenses")

train_dataset = train_dataset.astype('float32')
test_dataset  = test_dataset.astype('float32')

# convert pandas dataframes to tensorflow tensors
train_dataset = tf.cast(tf.convert_to_tensor(train_dataset.values), tf.float32)
test_dataset = tf.cast(tf.convert_to_tensor(test_dataset.values), tf.float32)

# build the model

In [ ]:
normalizer = layers.Normalization(axis=-1)
normalizer.adapt(train_dataset)


In [ ]:
#define a single feed-forward network regression

def build_model():
  model = keras.Sequential([
      normalizer,
      layers.Dense(128, activation='relu'),
      layers.Dense(128, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])
  model.compile(
      optimizer = keras.optimizers.Adam(learning_rate=0.0001),
      loss = "mse",
      metrics = ["mae", "mse"]
  )
  return model

model = build_model()
model.summary()

In [ ]:
# train the model
early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

history = model.fit(
    train_dataset, train_labels,
    validation_split=0.2,
    epochs=200,
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
# diagnose

plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Val MAE')
plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error')
plt.legend()
plt.show()

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
